In [88]:
#TSPLibのIdを揃える
import re

def load_id_mapping(mapping_file: str):
    id_mapping = {}
    with open(mapping_file, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or '->' not in line:
                continue
            try:
                old_id, new_id = line.split(' -> ')
                id_mapping[new_id.strip()] = old_id.strip()  # 逆に格納
            except ValueError:
                print(f"Skipping invalid line: {line}")
    return id_mapping

def replace_node_ids_in_tsplib(input_file: str, output_file: str, id_mapping: dict):
    with open(input_file, 'r') as f:
        content = f.readlines()
    
    replaced_content = []
    in_node_section = False
    for line in content:
        stripped_line = line.strip()
        
        if stripped_line == "NODE_COORD_SECTION":
            in_node_section = True
            replaced_content.append(line)
            continue
        
        if in_node_section:
            if stripped_line == "EOF":
                in_node_section = False
                replaced_content.append(line)
                continue
            
            parts = stripped_line.split()
            if len(parts) >= 3:
                node_id, x, y = parts[0], parts[1], parts[2]
                if node_id in id_mapping:
                    node_id = id_mapping[node_id]  # ノードIDを置換
                replaced_content.append(f"{node_id} {x} {y}\n")
                continue
        
        replaced_content.append(line)
    
    with open(output_file, 'w') as f:
        f.writelines(replaced_content)

# ID対応表のファイル
start_time = time.time()
mapping_file = "mappin90k_n5.txt"
id_mapping = load_id_mapping(mapping_file)

# 入出力ファイルのパス
input_tsplib_file = "similar_90K_n5.tsp"  # 変換前のTSPLIBファイル
output_tsplib_file = "similar_90K_n5_pre.tsp"  # 変換後のTSPLIBファイル

replace_node_ids_in_tsplib(input_tsplib_file, output_tsplib_file, id_mapping)
end_time = time.time()
print(f"Execution time: {end_time - start_time:.6f} seconds")

Execution time: 0.070955 seconds


In [93]:
#TourのIDを揃える

import time

def load_id_mapping(mapping_file: str):
    """ID変換辞書を作成（new_id -> old_id）"""
    id_mapping = {}
    with open(mapping_file, 'r') as f:
        for line in f:
            parts = line.strip().split(' -> ')
            if len(parts) == 2:
                old_id, new_id = parts
                id_mapping[new_id.strip()] = old_id.strip()
    return id_mapping

def replace_ids_in_tsplib(input_file: str, output_file: str, id_mapping: dict):
    """TSPLIBファイル内のIDを辞書を用いて高速変換"""
    with open(input_file, 'r') as f:
        with open(output_file, 'w') as out_f:
            for line in f:
                words = line.split()  # 単語単位で処理
                new_line = " ".join(id_mapping.get(word, word) for word in words)
                out_f.write(new_line + "\n")

# 実行部分
start_time = time.time()

mapping_file = "mappin90k_n5.txt"
input_tsplib_file = "updated_initial_90k_n5_3opt.tour"
output_tsplib_file = "updated_initial_90k_n5_pre_3opt.tour"

id_mapping = load_id_mapping(mapping_file)
replace_ids_in_tsplib(input_tsplib_file, output_tsplib_file, id_mapping)

end_time = time.time()
print(f"Execution time: {end_time - start_time:.6f} seconds")


Execution time: 0.053347 seconds
